In [5]:
import pandas as pd
import numpy as np
import warnings; warnings.simplefilter('ignore')
df = pd.read_csv("D:/r python/machine learning/data/Bike Share Demand/train.csv",parse_dates=['datetime'])
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
df['weekday']=df['datetime'].dt.weekday

df['season'] = df['season'].astype('category')
df['weather'] = df['weather'].astype('category')
df.drop(columns=['datetime','count', 'registered'],inplace=True)

dum_df = pd.get_dummies(df, drop_first=True)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled=scaler.fit_transform(dum_df)

from sklearn.cluster import KMeans

model = KMeans(n_clusters=4,random_state = 2019)

model.fit(df_scaled)
labels=pd.Series(model.predict(df_scaled)).rename('ClusterID')
Clustered=pd.concat([dum_df,labels],axis='columns')


In [8]:
SS0 = Clustered[Clustered['ClusterID']==0]
SS1 = Clustered[Clustered['ClusterID']==1]
SS2 = Clustered[Clustered['ClusterID']==2]
SS3 = Clustered[Clustered['ClusterID']==3]


SSx0 = SS0.drop(['ClusterID','casual'],axis='columns')
SSx1 = SS1.drop(['ClusterID','casual'],axis='columns')
SSx2 = SS2.drop(['ClusterID','casual'],axis='columns')
SSx3 = SS3.drop(['ClusterID','casual'],axis='columns')
SSy0=SS0['casual']
SSy1=SS1['casual']
SSy2=SS2['casual']
SSy3=SS3['casual']


In [9]:
from sklearn.ensemble import RandomForestRegressor

model_rf = RandomForestRegressor(random_state=2019,
                                  n_estimators=500,oob_score=True)
m1=model_rf.fit(SSx0,SSy0)
m2=model_rf.fit(SSx1,SSy1)
m3=model_rf.fit(SSx2,SSy2)
m4=model_rf.fit(SSx3,SSy3)


In [10]:
df2 = pd.read_csv("D:/r python/machine learning/data/Bike Share Demand/test.csv",parse_dates=['datetime'])
df2['year'] = df2['datetime'].dt.year
df2['month'] = df2['datetime'].dt.month
df2['day'] = df2['datetime'].dt.day
df2['hour'] = df2['datetime'].dt.hour
df2['weekday']=df2['datetime'].dt.weekday

df2['season'] = df2['season'].astype('category')
df2['weather'] = df2['weather'].astype('category')
datetimeOriginal = df2['datetime']
df2.drop(columns=['datetime'],inplace=True)

dum_df2 = pd.get_dummies(df2, drop_first=True)

X2 = dum_df2
X2scaled=scaler.fit_transform(X2)

model = KMeans(n_clusters=4,random_state = 2019)
model.fit(X2scaled)
labels2 = pd.Series( model.predict(X2scaled)).rename('ClusterID')

X2Clustered = pd.concat([X2,labels2,datetimeOriginal],axis='columns')
SS20 = X2Clustered[X2Clustered['ClusterID']==0]
SS21 = X2Clustered[X2Clustered['ClusterID']==1]
SS22 = X2Clustered[X2Clustered['ClusterID']==2]
SS23 = X2Clustered[X2Clustered['ClusterID']==3]

### For preserving the datetime sequences
SS20_datetimeOriginal = SS20['datetime']
SS21_datetimeOriginal = SS21['datetime']
SS22_datetimeOriginal = SS22['datetime']
SS23_datetimeOriginal = SS23['datetime']

SS20 = SS20.drop(['ClusterID','datetime'],axis='columns')
SS21 = SS21.drop(['ClusterID','datetime'],axis='columns')
SS22 = SS22.drop(['ClusterID','datetime'],axis='columns')
SS23 = SS23.drop(['ClusterID','datetime'],axis='columns')


In [11]:
y_predm1 = model_rf.predict(SS20)
y_predm1[y_predm1<0] = 0
y_predm2 = model_rf.predict(SS21)
y_predm2[y_predm2<0] = 0
y_predm3 = model_rf.predict(SS22)
y_predm3[y_predm3<0] = 0
y_predm4 = model_rf.predict(SS23)
y_predm4[y_predm4<0] = 0

In [12]:
clust0=pd.DataFrame({'datetime':SS20_datetimeOriginal,'casual':y_predm1})
clust1=pd.DataFrame({'datetime':SS21_datetimeOriginal,'casual':y_predm2})
clust2=pd.DataFrame({'datetime':SS22_datetimeOriginal,'casual':y_predm3})
clust3=pd.DataFrame({'datetime':SS23_datetimeOriginal,'casual':y_predm4})

In [ ]:
submit=pd.concat([clust0,clust1,clust2,clust3],ignore_index=True)
submit2=submit.sort_values(by=['datetime'])

submit2.to_csv("D:/r python/machine learning/data/Bike Share Demand/fi.csv",
              index=False)
